In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from copy import deepcopy
import warnings
import math


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

warnings.filterwarnings('ignore')

# Index variables

In [24]:
subject_number = 1

# Find where the static data end from data file 'SXXX_dynamic.csv'

In [25]:
static_end = []

for i in tqdm(range(10,10+subject_number)):
    current_dynamic = pd.read_csv('data_all_intact/S0' + str(i) + '_dynamic.csv', header=None)

    for j in range(len(current_dynamic)-1, -1, -1):
        if current_dynamic.iloc[j,0] == 0:
            static_end.append(j)
            break
    

In [26]:
static_end

[5413137]

# Analysis performance for different features

In [27]:
# save accuracy in array 'acc_features'
# save features name in array 'name_features'
acc_features = []
name_features = []

In [28]:
# STD, RMS, IEMG, MAV, WL, LOG, SSI, SKW, KURT, AAC, DASDV

## STD--Standard deviation

In [29]:
# index variable
'''change name, features'''
name = 'STD'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(trainx_raw.iloc[a-385:a+1, b].std())
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(testx_raw.iloc[a-385:a+1,b].std())
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

'change name, features'

In [40]:
pd.DataFrame(testx)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000030,0.000046,0.000027,0.000030,0.000029,0.000007,0.000015,7.946580e-08,0.000048,0.000029,0.000021,0.000011
1,0.000033,0.000053,0.000029,0.000036,0.000026,0.000007,0.000015,8.349542e-08,0.000056,0.000032,0.000023,0.000013
2,0.000039,0.000062,0.000028,0.000033,0.000024,0.000005,0.000017,9.162478e-08,0.000070,0.000028,0.000013,0.000025
3,0.000038,0.000070,0.000019,0.000025,0.000019,0.000004,0.000021,1.010973e-07,0.000077,0.000020,0.000012,0.000028
4,0.000034,0.000066,0.000013,0.000022,0.000013,0.000003,0.000022,1.022580e-07,0.000083,0.000017,0.000010,0.000038
...,...,...,...,...,...,...,...,...,...,...,...,...
5418,0.000018,0.000003,0.000004,0.000003,0.000002,0.000003,0.000003,1.311425e-07,0.000011,0.000008,0.000003,0.000003
5419,0.000015,0.000003,0.000004,0.000003,0.000002,0.000003,0.000003,1.262716e-07,0.000010,0.000008,0.000003,0.000002
5420,0.000016,0.000003,0.000004,0.000002,0.000002,0.000003,0.000003,1.290792e-07,0.000007,0.000008,0.000003,0.000003
5421,0.000016,0.000003,0.000004,0.000003,0.000002,0.000003,0.000003,1.358038e-07,0.000008,0.000007,0.000003,0.000003


In [43]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)

In [44]:
# train/test model
lightGBM_model = ltb.LGBMClassifier()
lightGBM_model.fit(trainx_df,trainy_df)
expected1 = testy_df.values.flatten()
predicted1 = lightGBM_model.predict(testx_df)
counter = 0
for i in range(0, len(expected1)):
    if expected1[i] == predicted1[i]:
        counter += 1

accuracy1 = counter/len(expected1)
accuracy1

LGBMClassifier()

0.7929190484971418

In [30]:
pd.DataFrame(trainx)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000003,0.000005,0.000005,0.000003,0.000003,0.000003,0.000003,1.028439e-07,0.000004,0.000007,0.000003,0.000003
1,0.000004,0.000005,0.000005,0.000003,0.000003,0.000003,0.000003,1.294892e-07,0.000004,0.000008,0.000003,0.000003
2,0.000004,0.000005,0.000005,0.000003,0.000003,0.000003,0.000003,1.290782e-07,0.000003,0.000008,0.000003,0.000003
3,0.000004,0.000005,0.000005,0.000003,0.000003,0.000003,0.000003,1.213311e-07,0.000003,0.000009,0.000003,0.000003
4,0.000003,0.000004,0.000005,0.000003,0.000003,0.000003,0.000003,1.411249e-07,0.000003,0.000009,0.000003,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...
16912,0.000023,0.000004,0.000008,0.000003,0.000003,0.000003,0.000002,9.136248e-08,0.000025,0.000015,0.000003,0.000003
16913,0.000024,0.000004,0.000008,0.000003,0.000003,0.000003,0.000003,9.405582e-08,0.000028,0.000016,0.000003,0.000003
16914,0.000024,0.000004,0.000008,0.000003,0.000002,0.000003,0.000003,8.494564e-08,0.000024,0.000015,0.000003,0.000003
16915,0.000023,0.000005,0.000007,0.000002,0.000002,0.000003,0.000003,8.796457e-08,0.000026,0.000014,0.000003,0.000003


In [31]:
pd.DataFrame(trainy)

,0
0,1
1,1
2,1
3,1
4,1
...,...
16912,0
16913,0
16914,0
16915,0
